## Imports

In [1]:
import pypsa
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os


## Read network file

In [28]:
# Function to read in the network file

def read_network_file():
    network = 'C:/Users/xenia/Desktop/Bolivia/pypsa-earth/networks/elec_s_4_ec_lcopt_Co2L-1H.nc'
    n = pypsa.Network(network)
    return n

network = 'C:/Users/xenia/Desktop/Bolivia/pypsa-earth/networks/elec_s_4_ec_lcopt_Co2L-1H.nc'
n = pypsa.Network(network)

# folder where the results from the previous year is saved
# scen_folder = 'results_RC_10'
# scen = 'RC_10'

# # Next year of the run
# year = 2035



INFO:pypsa.io:Imported network elec_s_4_ec_lcopt_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


## Scaling factors and Decomissioning

The scaling factors are used to increase the demand and decrease the emission constraint. The decomissioning (and comissioning, if applicable) is added on a yearly basis as well.

In [3]:
# Demand scaling factor - based on the initial demand in 2021

scale_demand = {
    2022: 1.039555791,
    2023: 1.037440228,
    2024: 1.030106894,
    2025: 1.029322163,
    2026: 1.051886792,
    2027: 1.044491339,
    2028: 1.044448186,
    2029: 1.044491013,
    2030: 1.044447874,
    2031: 1.044477281,
    2032: 1.044422438,
    2033: 1.040027091,
    2034: 1.039984371,
    2035: 1.040012523,
    'back':0.569269673, # scaling factor for scaling from 2035 to 2021
    'direct' : 1.7566367 # scaling factor from 2021 to 2035
}


In [4]:
# emissions decline

start_year = 2021
start_value = 2370000
end_year = 2035
end_value = 0

years = range(start_year, end_year + 1)
emissions = []

decrease_per_year = (start_value - end_value) / (end_year - start_year)

for i in years:
    emission = max(start_value - decrease_per_year * (i - start_year), end_value)
    emissions.append(emission)

# Make into dict
index = np.arange(2021,2036)
emission_limit = dict(zip(index, emissions))
emission_limit


{2021: 2370000.0,
 2022: 2200714.285714286,
 2023: 2031428.5714285714,
 2024: 1862142.8571428573,
 2025: 1692857.1428571427,
 2026: 1523571.4285714286,
 2027: 1354285.7142857143,
 2028: 1185000.0,
 2029: 1015714.2857142857,
 2030: 846428.5714285714,
 2031: 677142.857142857,
 2032: 507857.1428571427,
 2033: 338571.42857142864,
 2034: 169285.7142857141,
 2035: 0.0}

In [5]:
# decomissioning of powerplants

decom = {
2022 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.3, 'BO 2 OCGT': 36.76, 'BO 3 oil':0},
2023 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 36.77, 'BO 3 oil':0},
2024 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2025 : {'BO 0 OCGT':25.44, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2026 : {'BO 0 OCGT':0, 'BO 1 OCGT':17.86, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2027 : {'BO 0 OCGT':0, 'BO 1 OCGT':105.87, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2028 : {'BO 0 OCGT':0, 'BO 1 OCGT':16.9, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2029 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2030 : {'BO 0 OCGT':0, 'BO 1 OCGT':97.85, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2031 : {'BO 0 OCGT':0, 'BO 1 OCGT':109.37, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2032 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2033 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 0, 'BO 3 oil':2.16},
2034 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2035 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
}

# To force expansion, this dicts is used
expansion = {
    2022 : {'BO 0 geothermal':5, 'BO 0 solar':13, 'BO 2 ror':126},
    2023: { 'BO 2 solar':13, 'BO 3 ror':126},
    2024: {'BO 1 biomass': 21, 'BO 1 onwind':53, 'BO 0 solar':13, 'BO 2 ror':126},
    2025: { 'BO 1 onwind':24,'BO 2 solar':13, 'BO 3 ror':126},
    2026: {'BO 3 biomass': 21, 'BO 0 solar':13, 'BO 2 ror':126},
    2027: {'BO 0 geothermal':45, 'BO 1 onwind':25,'BO 2 solar':13, 'BO 3 ror':126},
    2028: {'BO 1 biomass': 21, 'BO 1 onwind':25, 'BO 0 solar':13, 'BO 2 ror':126},
    2029: {'BO 2 solar':13, 'BO 3 ror':126},
    2030: {'BO 3 biomass': 21, 'BO 1 onwind':35, 'BO 0 solar':13, 'BO 2 ror':126},
    2031: {},
    2032:{},
    2033:{},
    2034:{},
    2035:{},
}


# Nodes in the model (can change!!!!)
#0 : Southern
# 1: Oriental
#2: Central
#3: Northern

## All changes

This cell makes all changes that need to be done to run the next year all at once and saves it in a new file. 

In [20]:
# Function that implements all yearly changes

def yearly_changes(n,year):
    print(year)
    # ------- DEMAND ---------
    upscaling_factor = scale_demand[year]
    n.loads_t.p_set = n.loads_t.p_set * upscaling_factor
    #display(round((n.loads_t.p_set.sum().sum()/1000000),4))

    # ------- EMISSIONS -------
    n.global_constraints.constant = emission_limit[year]
    display(n.global_constraints.constant)



    # ------- GENERATOR EXTENSTION -----
    solved_network = f'C:/Users/xenia/Desktop/Bolivia/pypsa-earth/{scen_folder}/{scen}_{year-1}.nc'
    m = pypsa.Network(solved_network)
    additional_exp= m.generators.p_nom_opt - m.generators.p_nom
    # replace negative values with 0
    for index, value in additional_exp.items():
        if value < 0:
            additional_exp[index]=0
    # add expansion to previous network
    n.generators.p_nom = n.generators.p_nom.add(additional_exp, fill_value=0)
    n.generators.p_nom_min = n.generators.p_nom_min.add(additional_exp, fill_value=0)
    #display(n.generators.p_nom)


    # ------- STORES ----------
    additional_stores = m.stores.e_nom_opt - m.stores.e_nom
    additional_stores
    n.stores.e_nom = n.stores.e_nom.add(additional_stores, fill_value = 0)
    n.stores.e_nom_min = n.stores.e_nom_min.add(additional_stores, fill_value = 0)
    #display(n.stores.e_nom.sum())


    # ------- LINKS -------
    additional_links = m.links.p_nom_opt - m.links.p_nom
    additional_links
    n.links.p_nom = n.links.p_nom.add(additional_links, fill_value = 0)
    n.links.p_nom_min = n.links.p_nom_min.add(additional_links, fill_value = 0)
    #display(n.links.p_nom.sum())


    # ------- DECOM ---------
    for index,value in decom[year].items():
        n.generators.loc[index, 'p_nom'] = n.generators.loc[index].p_nom - value
        n.generators.loc[index, 'p_nom_min'] = n.generators.loc[index].p_nom_min - value

    # -------- EXPANSION ----------
    # for index,value in expansion[year].items():
    #     n.generators.loc[index, 'p_nom_min'] = n.generators.loc[index].p_nom_min + value

    # ------- HYDRO-DECOM-------- # is done seperately, because the  code is different
    if year == 2032:
        n.storage_units.loc['BO 2 hydro','p_nom'] = n.storage_units.loc['BO 2 hydro'].p_nom - 2.55

    # ------- SAVING ----------
    n.export_to_netcdf('C:/Users/xenia/Desktop/Bolivia/pypsa-earth/networks/elec_s_4_ec_lcopt_Co2L-1H.nc')

## Snakemake process

In [7]:
# Function that renames the results file and moves it to the scenario folder

def rename_results_file(year):
    current_location = 'C:/Users/xenia/Desktop/Bolivia/pypsa-earth/results/networks/'
    old_filename = "elec_s_4_ec_lcopt_Co2L-1H.nc"
    new_filename = f"{scen}_{year}.nc"
    new_location = f'C:/Users/xenia/Desktop/Bolivia/pypsa-earth/{scen_folder}/'

    # Create the full paths for the old and new files
    old_file_path = os.path.join(current_location, old_filename)
    new_file_path = os.path.join(new_location, new_filename)

    # Rename the file
    os.rename(old_file_path, new_file_path)


In [23]:
# determine scenario
scen_folder = 'results_RC_H2_new'
scen = 'RC_H2'

# !!!! Make sure the upscaling, emission constraints and decomissioning is run!!!

# run for loop over all years
for year in range(2021, 2036):
    n = read_network_file()
    if year == 2021:
        !snakemake -j 1 solve_all_networks
        rename_results_file(year)
    else:
        yearly_changes(n,year)
        !snakemake -j 1 solve_all_networks 
        rename_results_file(year)  

# And you're DONE!!! :D


INFO:pypsa.io:Imported network elec_s_4_ec_lcopt_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
Building DAG of jobs...
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                   count    min threads    max threads
------------------  -------  -------------  -------------
solve_all_networks        1              1              1
solve_network             1              1              1
total                     2              1              1

job                   count    min threads    max threads
------------------  -------  -------------  -------------
solve_all_networks        1              1              1
solve_network             1              1              1
total                     2              1              1

Select jobs to execute...

INFO:snakemake.logging:
[Mon Sep 18 22:39:58 2023]
INFO:snakemake.logging:[Mon Sep 18 22:39:58 20

Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-25
No parameters matching '_test' found
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-25
Read LP format model from file C:\Users\xenia\AppData\Local\Temp\pypsa-problem-anbpy0s_.lp
Reading time = 2.70 seconds
obj: 989885 rows, 490590 columns, 1883791 nonzeros
Set parameter Threads to value 4
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 1e-05
Set parameter AggFill to value 0
Set parameter PreDual to value 0
Set parameter GURO_PAR_BARDENSETHRESH to value 200
Set parameter LogFile to value "logs\solve_network\elec_s_4_ec_lcopt_Co2L-1H_solver.log"
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 4 threads

Optimize a model with 989885 rows, 490590 colum

INFO:pypsa.io:Imported network elec_s_4_ec_lcopt_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


2022


GlobalConstraint
CO2Limit    2.200714e+06
Name: constant, dtype: float64

...

2035


GlobalConstraint
CO2Limit    0.0
Name: constant, dtype: float64

INFO:pypsa.io:Imported network RC_ror_2034.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores
INFO:pypsa.io:Exported network elec_s_4_ec_lcopt_Co2L-1H.nc has loads, buses, carriers, links, lines, global_constraints, generators, stores, storage_units


Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-25
No parameters matching '_test' found
Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-25
Read LP format model from file C:\Users\xenia\AppData\Local\Temp\pypsa-problem-i2hyl46o.lp
Reading time = 2.76 seconds
obj: 989885 rows, 490590 columns, 1883791 nonzeros
Set parameter Threads to value 4
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 1e-05
Set parameter AggFill to value 0
Set parameter PreDual to value 0
Set parameter GURO_PAR_BARDENSETHRESH to value 200
Set parameter LogFile to value "logs\solve_network\elec_s_4_ec_lcopt_Co2L-1H_solver.log"
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: AMD Ryzen 7 4700U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 8 logical processors, using up to 4 threads

Optimize a model with 989885 rows, 490590 colum

Building DAG of jobs...
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                   count    min threads    max threads
------------------  -------  -------------  -------------
solve_all_networks        1              1              1
solve_network             1              1              1
total                     2              1              1

job                   count    min threads    max threads
------------------  -------  -------------  -------------
solve_all_networks        1              1              1
solve_network             1              1              1
total                     2              1              1

Select jobs to execute...

INFO:snakemake.logging:
[Mon Sep 18 22:56:31 2023]
INFO:snakemake.logging:[Mon Sep 18 22:56:31 2023]
rule solve_network:
    input: networks/elec_s_4_ec_lcopt_Co2L-1H.nc
    output: results/networks/elec_s_4_ec_lcopt_Co2L-1H.nc
    log: logs\solve_netwo

# Changes to the Network

Here all the general changes (except for expandabels, that's at the end of this file) that can be made to the model are shown. If more than the two last rules of the model are run, some of these changes may have to be done again. Use the "Checks" section to check your current network file. 

### Add Biomass in Node 0 and 3

The nodes may need to be adapted, depending on your model

In [18]:
n.add("Generator",
            'BO 0 biomass',
            bus='BO 0',
            carrier='biomass',
            p_nom=0,
            marginal_cost=n.generators.loc['BO 1 biomass'].marginal_cost,
            capital_cost = n.generators.loc['BO 1 biomass'].capital_cost,
            efficiency = n.generators.loc['BO 1 biomass'].efficiency,
            p_nom_extendable = n.generators.loc['BO 1 biomass'].p_nom_extendable,
            p_max_pu = n.generators.loc['BO 1 biomass'].p_max_pu,
            up_time_before= 1
            
)


n.add("Generator",
            'BO 2 biomass',
            bus='BO 2',
            carrier='biomass',
            p_nom=0,
            marginal_cost=n.generators.loc['BO 1 biomass'].marginal_cost,
            capital_cost = n.generators.loc['BO 1 biomass'].capital_cost,
            efficiency = n.generators.loc['BO 1 biomass'].efficiency,
            p_nom_extendable = n.generators.loc['BO 1 biomass'].p_nom_extendable,
            p_max_pu = n.generators.loc['BO 1 biomass'].p_max_pu,
            up_time_before= 1
            
)

### Add ROR 

Nodes may vary

In [10]:
n.storage_units

,bus,cyclic_state_of_charge,efficiency_dispatch,carrier,capital_cost,max_hours,p_min_pu,p_max_pu,p_nom,efficiency_store,...,marginal_cost_quadratic,build_year,lifetime,state_of_charge_initial,state_of_charge_initial_per_period,state_of_charge_set,cyclic_state_of_charge_per_period,standing_loss,inflow,p_nom_opt
StorageUnit,,,,,,,,,,,,,,,,,,,,,
BO 0 hydro,BO 0,True,0.9,hydro,160627.143522,6.0,0.0,0.5,26.50,0.0,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
BO 2 hydro,BO 2,True,0.9,hydro,160627.143522,6.0,0.0,0.5,428.47,0.0,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0
BO 3 hydro,BO 3,True,0.9,hydro,160627.143522,6.0,0.0,0.5,277.23,0.0,...,0.0,0,inf,0.0,False,NaN,True,0.0,0.0,0.0


In [11]:
n.add("Generator",
            'BO 2 ror',
            bus='BO 2',
            carrier='ror',
            p_nom=0,
            marginal_cost=0.021011,
            capital_cost = 270940.715283,
            efficiency = 0.9,
            p_nom_extendable = True,
            p_max_pu = 0.5,
            
)

n.add("Generator",
            'BO 3 ror',
            bus='BO 3',
            carrier='ror',
            p_nom=0,
            marginal_cost=0.021011,
            capital_cost = 270940.715283,
            efficiency = 0.9,
            p_nom_extendable = True,
            p_max_pu = 0.5,
            
)

### Biomas Max Capacity

The nodes may vary, the larger biomass max capacity is located in the Oriental zone

In [19]:
for index, row in n.generators.iterrows():
    # Check if the carrier is equal to "OCGT"
    if row['carrier'] == 'biomass':
        if index == 'BO 1 biomass':
        # Update the p_max_pu value to 0.64
            n.generators.at[index, 'p_nom_max'] = 357.13
        else:
            n.generators.at[index, 'p_nom_max'] = 21.01

n.generators.p_nom_max

Generator
BO 0 CCGT                   inf
BO 0 OCGT                   inf
BO 0 geothermal             inf
BO 0 onwind        6.051026e+05
BO 0 solar         7.058641e+05
BO 1 CCGT                   inf
BO 1 OCGT                   inf
BO 1 biomass       3.571300e+02
BO 1 onwind        6.224211e+05
BO 1 solar         2.391902e+05
BO 2 OCGT                   inf
BO 2 onwind        2.698194e+05
BO 2 solar         3.716044e+05
BO 3 OCGT                   inf
BO 3 biomass       2.101000e+01
BO 3 oil                    inf
BO 3 onwind        8.782172e+05
BO 3 solar         5.528653e+05
BO 0 biomass       2.101000e+01
BO 2 biomass       2.101000e+01
Name: p_nom_max, dtype: float64

### Battery max capacity

In [15]:
# discharger capacity

for index, row in n.links.iterrows():
    if row['carrier'] == 'battery discharger':
        n.links.at[index, 'p_nom_max'] = 225

display(n.links.p_nom_max)


# storage capacity

for index, row in n.stores.iterrows():
    if row['carrier'] == 'battery':
        n.stores.at[index, 'e_nom_max'] = 1350

n.stores.e_nom_max

Link
BO 0 battery charger         inf
BO 1 battery charger         inf
BO 2 battery charger         inf
BO 3 battery charger         inf
BO 0 battery discharger    225.0
BO 1 battery discharger    225.0
BO 2 battery discharger    225.0
BO 3 battery discharger    225.0
Name: p_nom_max, dtype: float64

Store
BO 0 battery    1350.0
BO 1 battery    1350.0
BO 2 battery    1350.0
BO 3 battery    1350.0
Name: e_nom_max, dtype: float64

### Capacity factors

With the new addition to model where the annual capacity factors can be set, this is not necessary anymore.

In [20]:
# Check the current status
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT          1.0
OCGT          1.0
biomass       1.0
geothermal    1.0
oil           1.0
onwind        1.0
solar         1.0
Name: p_max_pu, dtype: float64

In [21]:
# Iterate through the rows of the DataFrame
for index, row in n.generators.iterrows():
    # Check if the carrier is equal to "OCGT"
    if row['carrier'] == 'OCGT':
        # Update the p_max_pu value to 0.64
        n.generators.at[index, 'p_max_pu'] = 0.64 # 0.08 #0.64
    if row['carrier'] == 'CCGT':
        # Update the p_max_pu value to 0.64
        n.generators.at[index, 'p_max_pu'] = 0.64 #0.45 #0.64

    if row['carrier'] == 'biomass':
        # Update the p_max_pu value to 0.72
        n.generators.at[index, 'p_max_pu'] = 0.72 #0.32 #0.72 for the future 

        
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT          0.64
OCGT          0.64
biomass       0.72
geothermal    1.00
oil           1.00
onwind        1.00
solar         1.00
Name: p_max_pu, dtype: float64

# Save elec_s_4_ec_lcopt_Co2L-1H file

If you made changes to network file at the beginning, they need to be saved again. This code can be used for that. 

In [43]:
n.export_to_netcdf('C:/Users/xenia/Desktop/Bolivia/pypsa-earth/networks/elec_s_4_ec_lcopt_Co2L-1H.nc')

INFO:pypsa.io:Exported network elec_s_4_ec_lcopt_Co2L-1H.nc has loads, buses, carriers, links, lines, global_constraints, generators, stores, storage_units


<xarray.Dataset>
Dimensions:                               (snapshots: 8760,
                                           investment_periods: 0, loads_i: 4,
                                           loads_t_p_set_i: 4, buses_i: 12,
                                           carriers_i: 12, links_i: 16,
                                           lines_i: 3, global_constraints_i: 1,
                                           generators_i: 19,
                                           generators_t_p_max_pu_i: 8,
                                           stores_i: 8, storage_units_i: 3,
                                           storage_units_t_inflow_i: 3)
Coordinates: (12/14)
  * snapshots                             (snapshots) int64 0 1 2 ... 8758 8759
  * investment_periods                    (investment_periods) int64 
  * loads_i                               (loads_i) object 'BO 0' ... 'BO 3'
  * loads_t_p_set_i                       (loads_t_p_set_i) object 'BO 0' ......
  * buses_i                               (buses_i) object 'BO 0' ... 'BO 3 b...
  * carriers_i                            (carriers_i) object 'biomass' ... '...
    ...                                    ...
  * global_constraints_i                  (global_constraints_i) object 'CO2L...
  * generators_i                          (generators_i) object 'BO 0 CCGT' ....
  * generators_t_p_max_pu_i               (generators_t_p_max_pu_i) object 'B...
  * stores_i                              (stores_i) object 'BO 0 H2' ... 'BO...
  * storage_units_i                       (storage_units_i) object 'BO 0 hydr...
  * storage_units_t_inflow_i              (storage_units_t_inflow_i) object '...
Data variables: (12/67)
    snapshots_snapshot                    (snapshots) datetime64[ns] 2013-01-...
    snapshots_objective                   (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_generators                  (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_stores                      (snapshots) float64 1.0 1.0 ... 1.0
    investment_periods_objective          (investment_periods) float64 
    investment_periods_years              (investment_periods) float64 
    ...                                    ...
    storage_units_carrier                 (storage_units_i) object 'hydro' .....
    storage_units_capital_cost            (storage_units_i) float64 1.606e+05...
    storage_units_efficiency_dispatch     (storage_units_i) float64 0.9 0.9 0.9
    storage_units_p_max_pu                (storage_units_i) float64 0.5 0.5 0.5
    storage_units_p_min_pu                (storage_units_i) float64 0.0 0.0 0.0
    storage_units_t_inflow                (snapshots, storage_units_t_inflow_i) float64 ...
Attributes:
    network_name:           
    network_pypsa_version:  0.24.0
    network_srid:           4326
    meta:                   {"version": "0.2.2", "tutorial": false, "logging"...

# Checks

If you don't know the setting of your current network file, use the following cells to check it. 

In [29]:
# Which generators do exist?

n.generators

,p_nom_max,weight,p_nom,capital_cost,efficiency,p_nom_min,marginal_cost,p_nom_extendable,bus,carrier,...,sign,marginal_cost_quadratic,lifetime,committable,start_up_cost,shut_down_cost,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Generator,,,,,,,,,,,,,,,,,,,,,
BO 0 CCGT,inf,0.000000,454.30,84469.122809,0.500,454.30,12.240000,False,BO 0,CCGT,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
BO 0 OCGT,inf,0.000000,34.50,47234.561404,0.390,34.50,13.564103,False,BO 0,OCGT,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
BO 0 geothermal,inf,0.000000,5.00,334431.013779,0.239,5.00,0.000000,True,BO 0,geothermal,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
BO 0 onwind,6.051026e+05,41405.519294,0.00,109295.569252,1.000,0.00,0.015000,True,BO 0,onwind,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
BO 0 solar,7.058641e+05,139220.197844,64.70,50855.958267,1.000,64.70,0.010000,True,BO 0,solar,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
BO 1 CCGT,inf,0.000000,909.69,84469.122809,0.500,909.69,12.240000,False,BO 1,CCGT,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
BO 1 OCGT,inf,0.000000,749.76,47234.561404,0.390,749.76,13.564103,False,BO 1,OCGT,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
BO 1 biomass,3.571300e+02,0.000000,60.64,278015.359506,0.468,60.64,2.136752,True,BO 1,biomass,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0
BO 1 onwind,6.224211e+05,71313.098136,100.80,109295.569252,1.000,100.80,0.015000,True,BO 1,onwind,...,1.0,0.0,inf,False,0.0,0.0,NaN,NaN,1.0,1.0


In [35]:
# Which generators are currently extendable?

n.generators.p_nom_extendable


Generator
BO 0 CCGT       False
BO 0 OCGT       False
BO 0 onwind      True
BO 0 solar       True
BO 1 CCGT       False
BO 1 OCGT       False
BO 1 biomass     True
BO 1 onwind      True
BO 1 solar       True
BO 2 OCGT       False
BO 2 onwind      True
BO 2 solar       True
BO 3 OCGT       False
BO 3 biomass     True
BO 3 oil        False
BO 3 onwind      True
BO 3 solar       True
BO 0 biomass     True
BO 2 biomass     True
Name: p_nom_extendable, dtype: bool

In [36]:
# Check the current status of the cp
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT       0.64
OCGT       0.64
biomass    0.72
oil        1.00
onwind     1.00
solar      1.00
Name: p_max_pu, dtype: float64

In [37]:
# Which generators have which expansion limit?

n.generators.p_nom_max

Generator
BO 0 CCGT                inf
BO 0 OCGT                inf
BO 0 onwind     6.051026e+05
BO 0 solar      7.058641e+05
BO 1 CCGT                inf
BO 1 OCGT                inf
BO 1 biomass    3.571300e+02
BO 1 onwind     6.224211e+05
BO 1 solar      2.391902e+05
BO 2 OCGT                inf
BO 2 onwind     2.698194e+05
BO 2 solar      3.716044e+05
BO 3 OCGT                inf
BO 3 biomass    2.101000e+01
BO 3 oil                 inf
BO 3 onwind     8.782172e+05
BO 3 solar      5.528653e+05
BO 0 biomass    2.101000e+01
BO 2 biomass    2.101000e+01
Name: p_nom_max, dtype: float64

In [38]:
# Which links have which expansion limit?

n.links.p_nom_max

Link
BO 0 H2 Electrolysis       inf
BO 1 H2 Electrolysis       inf
BO 2 H2 Electrolysis       inf
BO 3 H2 Electrolysis       inf
BO 0 H2 Fuel Cell          inf
BO 1 H2 Fuel Cell          inf
BO 2 H2 Fuel Cell          inf
BO 3 H2 Fuel Cell          inf
BO 0 battery charger       inf
BO 1 battery charger       inf
BO 2 battery charger       inf
BO 3 battery charger       inf
BO 0 battery discharger    inf
BO 1 battery discharger    inf
BO 2 battery discharger    inf
BO 3 battery discharger    inf
Name: p_nom_max, dtype: float64

In [39]:
# What is the sum of the demand in the network?

round(n.loads_t.p_set.sum().sum()/1000000,8)

9.45507556

In [40]:
# Is there any installed storage capacity?

n.stores.e_nom

Store
BO 0 H2         0.0
BO 1 H2         0.0
BO 2 H2         0.0
BO 3 H2         0.0
BO 0 battery    0.0
BO 1 battery    0.0
BO 2 battery    0.0
BO 3 battery    0.0
Name: e_nom, dtype: float64

In [41]:
# What is the current emission limit?

n.global_constraints.constant.loc['CO2Limit']

2370000.0

In [42]:
# Which stores have a maximum expansion limit?

n.stores.e_nom_max

Store
BO 0 H2         inf
BO 1 H2         inf
BO 2 H2         inf
BO 3 H2         inf
BO 0 battery    inf
BO 1 battery    inf
BO 2 battery    inf
BO 3 battery    inf
Name: e_nom_max, dtype: float64

# Restoring to 2021

When you reached your final year, you can easily restore your network to the year 2021. It is recommended, though, to save the initial 2021 file at the beginning. 

## Demand

In [84]:
year = 'back'
upscaling_factor = scale_demand[year]   # 0.569269674



n.loads_t.p_set = n.loads_t.p_set * upscaling_factor

year = 2021

In [85]:
n.loads_t.p_set.sum().sum()

9455075.555625789

## Emission limit

In [41]:
n.global_constraints.constant = 1000000000 #emission_limit[2021]
n.global_constraints.constant

GlobalConstraint
CO2Limit    1000000000
Name: constant, dtype: int64

## Expansions

In [86]:
solved_network = f'C:/Users/xenia/Desktop/Bolivia/pypsa-earth/results_NS/NS_2021.nc'
m = pypsa.Network(solved_network)

n.generators.p_nom = m.generators.p_nom
n.generators.p_nom

INFO:pypsa.io:Imported network NS_2021.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


Generator
BO 0 CCGT          454.30
BO 0 OCGT           34.50
BO 0 geothermal      0.00
BO 0 onwind          0.00
BO 0 solar          64.70
BO 1 CCGT          909.69
BO 1 OCGT          749.76
BO 1 biomass        60.64
BO 1 onwind        100.80
BO 1 solar           0.00
BO 2 OCGT          115.41
BO 2 onwind         26.54
BO 2 solar         100.20
BO 3 OCGT           49.25
BO 3 biomass         4.94
BO 3 oil            31.32
BO 3 onwind          0.00
BO 3 solar           0.00
BO 2 ror             0.00
BO 3 ror             0.00
BO 0 biomass         0.00
BO 2 biomass         0.00
Name: p_nom, dtype: float64

In [87]:
n.generators.p_nom_min = m.generators.p_nom_min
n.generators.p_nom_min

Generator
BO 0 CCGT          454.30
BO 0 OCGT           34.50
BO 0 geothermal      0.00
BO 0 onwind          0.00
BO 0 solar          64.70
BO 1 CCGT          909.69
BO 1 OCGT          749.76
BO 1 biomass        60.64
BO 1 onwind        100.80
BO 1 solar           0.00
BO 2 OCGT          115.41
BO 2 onwind         26.54
BO 2 solar         100.20
BO 3 OCGT           49.25
BO 3 biomass         4.94
BO 3 oil            31.32
BO 3 onwind          0.00
BO 3 solar           0.00
BO 2 ror             0.00
BO 3 ror             0.00
BO 0 biomass         0.00
BO 2 biomass         0.00
Name: p_nom_min, dtype: float64

In [10]:
# new biomass to 0
for index, row in n.generators.iterrows():
    if index == 'BO 0 biomass':
        n.generators.at[index, 'p_nom'] = 0
    if index == 'BO 2 biomass':
            n.generators.at[index, 'p_nom'] = 0
n.generators.p_nom

Generator
BO 0 CCGT       454.30
BO 0 OCGT        34.50
BO 0 onwind       0.00
BO 0 solar       64.70
BO 1 OCGT        49.25
BO 1 biomass      4.94
BO 1 oil         31.32
BO 1 onwind       0.00
BO 1 solar        0.00
BO 2 CCGT       909.69
BO 2 OCGT       749.76
BO 2 biomass     60.64
BO 2 onwind     100.80
BO 2 solar        0.00
BO 3 OCGT       115.41
BO 3 onwind      26.54
BO 3 solar      100.20
BO 0 biomass      0.00
BO 3 biomass      0.00
Name: p_nom, dtype: float64

In [88]:
n.stores.e_nom = 0

n.stores.e_nom.sum()

0

In [89]:
n.links.p_nom = 0
n.links.p_nom.sum()

0

# Changing the expandabel carriers

This can also be done in the config file, but doing it here saves you from running the model again and have to do all the other changes again as well.

### RES to TRUE

In [17]:
# Iterate through the rows of the DataFrame
for index, row in n.generators.iterrows():
   
    if row['carrier'] == 'biomass':
        
        n.generators.at[index, 'p_nom_extendable'] = True 
    if row['carrier'] == 'solar':
        
        n.generators.at[index, 'p_nom_extendable'] = True 

    if row['carrier'] == 'onwind':
       
        n.generators.at[index, 'p_nom_extendable'] = True 

### Fossils to FALSE

In [12]:
for index, row in n.generators.iterrows():
   
    if row['carrier'] == 'OCGT':
        n.generators.at[index, 'p_nom_extendable'] = False 

    if row['carrier'] == 'CCGT':
        
        n.generators.at[index, 'p_nom_extendable'] = False 

    if row['carrier'] == 'oil':
       
        n.generators.at[index, 'p_nom_extendable'] = False 

n.generators.p_nom_extendable

Generator
BO 0 CCGT          False
BO 0 OCGT          False
BO 0 geothermal     True
BO 0 onwind         True
BO 0 solar          True
BO 1 CCGT          False
BO 1 OCGT          False
BO 1 biomass        True
BO 1 onwind         True
BO 1 solar          True
BO 2 OCGT          False
BO 2 onwind         True
BO 2 solar          True
BO 3 OCGT          False
BO 3 biomass        True
BO 3 oil           False
BO 3 onwind         True
BO 3 solar          True
Name: p_nom_extendable, dtype: bool